Install the libraries

In [ ]:
!pip install -q langchain transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 110.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 90.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.9 MB/s eta 0:00:00


Load the libraries

In [ ]:
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain


from transformers import AutoModel
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

import json
import textwrap


Download the Model - We are using NousResearch's Llama2 which is the same as Meta AI's Llama 2, the only difference being "**Not requiring authentication to download**"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")

In [ ]:
model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             load_in_4bit=True,
                                             bnb_4bit_quant_type="nf4",
                                             bnb_4bit_compute_dtype=torch.float16)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


Define Transformers Pipeline which will be fed into Langchain

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.float16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )


Define the Prompt format for Llama 2 - This might change if you have a different model

In [ ]:

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a finance assistant and retirement planning expert, excelling at providing advice and guidance for financial decisions and retirement planning.
Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.Just say you don't know and you are sorry!"""


All the helper fucntions to generate prompt, prompt template, clean up output text

In [ ]:

def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT, citation=None):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST

    if citation:
        prompt_template += f"\n\nCitation: {citation}"  # Insert citation here

    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")

def generate(text, citation=None):
    prompt = get_prompt(text, citation=citation)
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(**inputs,
                                 max_length=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs

def parse_text(text):
    wrapped_text = textwrap.fill(text, width=100)
    print(wrapped_text + '\n\n')

Defining Langchain LLM

In [ ]:

llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0.7,'max_length': 256, 'top_k' :50})



In [ ]:
system_prompt = "You are a finance assistant and retirement planning expert, excelling at providing advice and guidance for financial decisions and retirement planning "
instruction = "Convert the following input text from a stupid human to a well-reasoned advice in paragraph form:\n\n {text}"
template = get_prompt(instruction, system_prompt)
print(template)


[INST]<>
You are a finance assistant and retirement planning expert, excelling at providing advice and guidance for financial decisions and retirement planning 
<>

Convert the following input text from a stupid human to a well-reasoned and step-by-step throughout advice:

 {text}[/INST]


In [ ]:
prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm, verbose = False)



In [ ]:
text = "When is the right time to retire"

In [ ]:
response = llm_chain.run(text)
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


  Ah, a question that gets to the heart of many people's financial and personal goals! The right time to retire can vary depending on your individual circumstances, but here's a step-by-step guide to help you plan and prepare for your retirement:

Step 1: Determine Your Retirement Goals and Timeline

* Start by thinking about what you want your retirement to look like. Do you want to travel, pursue hobbies, or volunteer? Do you want to continue working part-time or take a full break?
* Next, estimate your financial goals for retirement, including how much money you'll need to cover your living expenses, medical costs, and other expenses.
* Consider your desired retirement age and work backward to determine when you'll need to start saving enough to support your retirement lifestyle.

Step 2: Build Your Retirement Nest Egg

* To build your retirement nest egg, start by contributing to a 401(k) or IRA, if your employer offers it, or consider opening a Roth or traditional IRA.
* Take adva

In [ ]:
text = "How much money i need to retire?"

In [ ]:
response = llm_chain.run(text)
print(response)

  Ah, a very astute question! 😊 Retirement planning is an essential aspect of personal finance, and determining how much money you need to retire can be a bit complex. However, with my expertise and your willingness to follow a few simple steps, we can come up with a reasonable estimate that will help you achieve your retirement goals. 🎉

Step 1: Calculate Your Future Expenses
To determine how much money you need to retire, we need to estimate your future expenses during retirement. A general rule of thumb is to plan for a 30% to 50% reduction in living expenses after retirement. This means you'll need to account for inflation, healthcare costs, and other expenses that may arise.

Can you tell me about your current living expenses? This will give us a baseline to work with. 🤔

Step 2: Estimate Your Retirement Income
The next step is to estimate the amount of income you'll have available during retirement. This can come from various sources, such as your retirement accounts, social secu